In [2]:

from text_preprocessing import text_processing

import string
import sys

from pathlib import Path
from gensim import corpora, models
import pandas as pd
from nltk import WordNetLemmatizer
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import scipy.cluster.hierarchy as sch
from nltk.corpus import *
from nltk.tokenize import *
import contractions
from sklearn.feature_extraction import *
from sklearn.metrics import *
import re
import nltk



def return_dict_corpus(train_df):
    texts = list(train_df.lemmatize_title_w_pos.values)
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    print(f'number of unique tokens: {len(dictionary)}')
    print(f'number of documents : {len(corpus)}')

    return texts, dictionary, corpus


path = Path(r'../Tawosi_Dataset')

train, valid, test = pd.read_csv(path / 'DM-train.csv'), pd.read_csv(path / 'DM-valid.csv'), pd.read_csv(
    path / 'DM-test.csv')

data = pd.concat([train, valid])

texts = []

# print(train)

train = text_processing(train)

texts, dictionary, corpus = return_dict_corpus(train)

num_topics = 20

lda_model = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    random_state=100,
    chunksize=2000,
    iterations=400,
    passes=20,
    per_word_topics=True,
    alpha='auto',
    eta='auto',
    eval_every=True
)

topics = lda_model[corpus]

train_ik = train.issuekey.values


test_prob = {}

topics_number = set(f'topic_{i}' for i in range(num_topics))

for key, prob in zip(train_ik, topics):
    top_preds = {}

    for (topic_no, value) in prob[0]:
        top_preds[f'topic_{topic_no}'] = value


    for tn in topics_number:
        if not top_preds.get(tn, None):
            top_preds[tn] = 0.0

    test_prob[key] = top_preds


prob_df_cols = sorted(list(topics_number), key=lambda x: int(x.split('_')[1]))

prob_df = pd.DataFrame.from_dict(test_prob, orient='index')

prob_df.index.name = 'issuekey'

prob_df.to_csv('prob_df.csv')

dendrogram = sch.dendrogram(sch.linkage(prob_df.values, method='ward'), no_plot=True)

cn = len(set(dendrogram['color_list'])) - 1

print(f"cluster no: {cn}")

ac_m = AgglomerativeClustering(n_clusters=cn, affinity='euclidean', linkage='ward')

preds = ac_m.fit_predict(prob_df.values)

prob_df['labels'] = preds

print(prob_df.labels.value_counts())

/home/sirakr/miniconda3/envs/fastai_l/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


number of unique tokens: 4183
number of documents : 3227
cluster no: 9
0    1508
1     327
7     246
4     222
3     211
2     194
5     190
6     187
8     142
Name: labels, dtype: int64


In [4]:
test_df = text_processing(test)
t_texts, t_dictionary, t_corpus = return_dict_corpus(test)

number of unique tokens: 1936
number of documents : 1078


In [5]:
t_topics = lda_model[t_corpus[0]]

In [13]:
t_topics

([(3, 0.17525354),
  (4, 0.5068754),
  (6, 0.010642398),
  (11, 0.010100877),
  (16, 0.010015527),
  (18, 0.17296453)],
 [(0, [4]), (1, [4]), (2, [4]), (3, [18]), (4, [3])],
 [(0, [(4, 0.9999258)]),
  (1, [(4, 0.9999708)]),
  (2, [(4, 0.9997431)]),
  (3, [(18, 0.9996705)]),
  (4, [(3, 0.99963593)])])

In [14]:
def return_prob_list(probs):
    ind_prob = {}

    for (topic_no, prob) in probs[0]:
        ind_prob[f'topic_{topic_no}'] = prob

    for tn in topics_number:
        if not ind_prob.get(tn, None): ind_prob[tn] = 0.0

    return ind_prob


In [15]:
return_prob_list(t_topics)

{'topic_3': 0.17525354,
 'topic_4': 0.5068754,
 'topic_6': 0.010642398,
 'topic_11': 0.010100877,
 'topic_16': 0.010015527,
 'topic_18': 0.17296453,
 'topic_12': 0.0,
 'topic_19': 0.0,
 'topic_15': 0.0,
 'topic_9': 0.0,
 'topic_14': 0.0,
 'topic_10': 0.0,
 'topic_0': 0.0,
 'topic_17': 0.0,
 'topic_13': 0.0,
 'topic_1': 0.0,
 'topic_2': 0.0,
 'topic_5': 0.0,
 'topic_7': 0.0,
 'topic_8': 0.0}

In [26]:
inp = list(dict(sorted(return_prob_list(t_topics).items(), key= lambda item: int(item[0].split('_')[1]))).values())

In [47]:
inps = {
    'inp-6767': return_prob_list(t_topics)
}

inp_df = pd.DataFrame.from_dict(inps, orient='index')
inp_df.index.name = 'issue_key'

In [48]:
# ac_m.fit_predict(inp_df.values)

In [49]:
inp_df.values

array([[0.17525354, 0.5068754 , 0.0106424 , 0.01010088, 0.01001553,
        0.17296453, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]])

In [59]:
ac_m.fit_predict(inp_df.values)

ValueError: Found array with 1 sample(s) (shape=(1, 20)) while a minimum of 2 is required by AgglomerativeClustering.

In [54]:
prob_df.shape

(3227, 21)

In [55]:
inp_df.shape

(1, 20)

In [65]:
ac_m.fit_predict(inp_df.to_numpy().reshape(-1,1))


array([4, 7, 5, 6, 8, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1])

In [66]:
prob_df

,topic_3,topic_4,topic_6,topic_11,topic_16,topic_18,topic_12,topic_19,topic_15,topic_9,...,topic_10,topic_0,topic_17,topic_13,topic_1,topic_2,topic_5,topic_7,topic_8,labels
issuekey,,,,,,,,,,,,,,,,,,,,,
DM-9,0.175254,0.506875,0.010642,0.010101,0.010016,0.172965,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
DM-71,0.011095,0.010498,0.012759,0.012110,0.211073,0.000000,0.010207,0.011319,0.000000,0.010514,...,0.011525,0.000000,0.000000,0.000000,0.607230,0.000000,0.000000,0.000000,0.000000,1
DM-68,0.000000,0.423613,0.010641,0.010100,0.010015,0.000000,0.000000,0.212937,0.000000,0.000000,...,0.221396,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6
DM-52,0.330720,0.242236,0.012760,0.012110,0.256973,0.000000,0.010207,0.011319,0.000000,0.010514,...,0.011526,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4
DM-78,0.000000,0.000000,0.010642,0.010101,0.010015,0.837085,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DM-16286,0.116978,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.560591,0.000000,0.000000,0.000000,0.227189,0.000000,0.000000,0.000000,0.000000,2
DM-18223,0.000000,0.000000,0.000000,0.000000,0.000000,0.129856,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.141204,0.000000,0.000000,0.000000,0.254563,0
DM-18209,0.000000,0.000000,0.000000,0.000000,0.154350,0.148349,0.000000,0.000000,0.000000,0.000000,...,0.150610,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8
